In [114]:
import numpy as np
import pandas as pd

In [115]:
data=pd.read_csv("dataset.csv")


In [116]:
#print(data[data['I_W'].isnull()])

In [117]:
#data.dtypes

In [118]:
#data['O_Size'].value_counts()

In [119]:
#data['I_Id'].value_counts()

In [120]:
#data['I_W'].value_counts()

In [121]:
#data['I_Fat_C'].value_counts()

In [122]:
del data['I_MRP_JD']

In [123]:
#data.head(20)

In [124]:
data['I_Fat_C'] = data['I_Fat_C'].replace(['LF'], 'Low Fat')
data['I_Fat_C'] = data['I_Fat_C'].replace(['reg'], 'Regular')
data['I_Fat_C'] = data['I_Fat_C'].replace(['low fat'], 'Low Fat')
data['I_W'] = data.groupby('I_Id')['I_W'].transform(lambda x: x.fillna(x.mode(False)[0]))
data['I_W'] = data['I_W'].where(data['I_Id']!='FDH52', 9.42, axis=0)
data['I_W'] = data['I_W'].where(data['I_Id']!='NCU29', 7.685, axis=0)
data['I_W'] = data['I_W'].where(data['I_Id']!='NCT53', 5.4, axis=0)
data['I_W']=data['I_W'].fillna(data['I_W'].mean)
probability = data['O_Size'].value_counts() / data['O_Size'].count()
data['O_Size'] = data['O_Size'].fillna(pd.Series(np.random.choice(list(probability.index), p=probability, size=len(data))))
data.isnull().sum()
data['I_W'] = pd.factorize(data['I_W'])[0]


In [125]:
from sklearn.preprocessing import LabelEncoder
cat_columns= data.select_dtypes(['object']).columns
label_encoder=LabelEncoder()
data[cat_columns]=data[cat_columns].apply(LabelEncoder().fit_transform)
data.head()

,I_Id,I_W,I_Fat_C,I_Vis,I_category,I_MRP_US,O_Id,O_Establ_Y,O_Size,I_Recalled,O_Loc_T,O_T,I_O_Sales
0,156,0,0,0.016172,4,317.8092,9,2009,1,0,0,1,3915.1380
1,8,1,1,0.019403,14,116.2692,3,2019,1,1,2,0,623.4228
2,662,2,0,0.016885,10,209.6180,9,2009,1,0,0,1,2277.2700
3,1121,3,1,0.000000,6,250.0950,0,2008,1,0,2,3,912.3800
4,1297,4,0,0.000000,9,121.8614,1,1997,0,0,2,1,1174.7052


In [126]:
from sklearn.model_selection import train_test_split
X=data.drop(['O_T'],axis=1)
y=data['O_T']
X_train , X_test, y_train, y_test =train_test_split(X,y,test_size=0.33,random_state=42)

In [127]:
X_train_zscore = (X_train - X_train.mean()) / X_train.std()
X_test_zscore = (X_test - X_test.mean()) / X_test.std()

In [128]:
def algorithm2(y_train,X_train,X_test):
  alg=[]
  for i in range(X_test.shape[0]):
    distances=np.linalg.norm(X_train - X_test.iloc[i], axis=1)
    k=3
    nearest_neighbour_ids=distances.argsort()[:k]
    nearest_neighbour_rings= y_train.iloc[nearest_neighbour_ids]
    alg.append(nearest_neighbour_rings.mode()[0])
  return alg

In [129]:
y_pred=algorithm2(y_train,X_train_zscore,X_test_zscore)

In [130]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score

In [131]:
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("Precision: ",precision_score(y_test,y_pred, average = 'macro'))
print("Recall: ",recall_score(y_test,y_pred, average = 'macro'))
print("F1_score: ",f1_score(y_test,y_pred, average = 'macro'))

Accuracy:  0.9943121222893708
Precision:  0.9910785872177352
Recall:  0.992502499891309
F1_score:  0.9916910758913449


In [132]:
feature_range = (0,1)
min = X_train.min(axis=0)
scale = (X_train.max(axis=0) - X_train.min(axis = 0)) / (feature_range[1] - feature_range[0])

X_train_minmax = (X_train - min) / scale
min = X_test.min(axis=0)
scale = (X_test.max(axis=0) - X_test.min(axis = 0)) / (feature_range[1] - feature_range[0])
X_test_minmax = (X_test - min) / scale

In [133]:
y_pred=algorithm2(y_train,X_train_minmax,X_test_minmax)

In [134]:
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("Precision: ",precision_score(y_test,y_pred, average = 'macro'))
print("Recall: ",recall_score(y_test,y_pred, average = 'macro'))
print("F1_score: ",f1_score(y_test,y_pred, average = 'macro'))

Accuracy:  0.9975115535015997
Precision:  0.9983618126589887
Recall:  0.9953208556149733
F1_score:  0.996817888731675


In [135]:
y_pred=algorithm2(y_train,X_train,X_test)

In [136]:
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("Precision: ",precision_score(y_test,y_pred, average = 'macro'))
print("Recall: ",recall_score(y_test,y_pred, average = 'macro'))
print("F1_score: ",f1_score(y_test,y_pred, average = 'macro'))

Accuracy:  0.6718805545680768
Precision:  0.5068066395363016
Recall:  0.46869204928457997
F1_score:  0.47698474696511683
